In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [4]:
# Crear carpeta "images" si no existe
if not os.path.exists("images"):
    os.makedirs("images")

In [ ]:
pd.set_option('display.max_columns', None) # es una configuración de la librería Pandas que controla cómo se visualizan los DataFrames cuando los imprimes en la pantalla

# 'display.max_columns' le dice a Pandas que estás configurando el número máximo de columnas que debe mostrar cuando imprime un DataFrame.
# El Valor: None Al establecer el valor en None, le estás diciendo a Pandas que desactive el límite.
# Quiero que me muestres TODAS las columnas de mi DataFrame, sin importar cuántas tenga, en lugar de ocultar las del medio y poner puntos suspensivos (...)."
# Es una herramienta esencial para el Análisis Exploratorio de Datos (EDA), ya que te ayuda a ver la "foto completa" de tu conjunto de datos

## Carga y exploración inicial:

In [ ]:
# En este caso no es adecuado usar el index_col = 0, porque para la función merge que tendré que usar para unir los csv, necesito que 'loyalty number' sea una columna regular
df_history = pd.read_csv("data/Customer Loyalty History.csv") 
#primera exploración visual
df_history

In [ ]:
df_activity = pd.read_csv("data/Customer Flight Activity.csv")
df_activity

In [6]:
def archivar_grafica(nombre):
    ruta = f"images/{nombre}.png"
    plt.savefig(ruta, dpi=300, bbox_inches="tight")
    print(f" Gráfica archivada en: {ruta}")

In [ ]:
# visual rápida de los 2 df: 

df_history

In [ ]:
# NO EJECUTAR AÚN! 
# FASE DE VISUALIZACIONES (pegar esto cada vez que haga una gráfica):

# código para pegar al crear gráfica y que se ejecuten conjuntamente, para archivar directamente la imagen.
plt.figure(figsize=(10,6))
sns.barplot(data=df, x="Month", y="Flights Booked")
plt.title("Vuelos por mes")

plt.plot(x, y)
archivar_grafica("grafica_ejemplo")
plt.show()